# Import Basic Packages

In [6]:
import pandas as pd
import numpy as np 
import re
import time

# Import All NLP Packages

In [58]:
import unidecode
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller
import string
import timeit
from bs4 import BeautifulSoup

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


# Load Dataset

In [22]:
Df = pd.read_csv('train.tsv', sep='\t')
print(Df.Sentiment.value_counts())
df = pd.concat([Df[Df.Sentiment==0], Df[Df.Sentiment==4].sample(7072)])[["Phrase","Sentiment"]]
Df=df
Df.head()
# Df = pd.read_csv('New Task.csv', encoding = 'latin-1')
print('Number of Data points : ', Df.shape[0])
print('Number of features :', Df.shape[1])
print('features :', Df.columns.values)
# Show Dataset
Df.head()
Df

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64
Number of Data points :  14144
Number of features : 2
features : ['Phrase' 'Sentiment']


,Phrase,Sentiment
101,would have a hard time sitting through this one,0
103,have a hard time sitting through this one,0
157,Aggressive self-glorification and a manipulati...,0
159,self-glorification and a manipulative whitewash,0
201,Trouble Every Day is a plodding mess .,0
...,...,...
18320,most aggressive and most sincere,4
73944,greatest date movies,4
137657,a sharp script and strong performances,4
89738,"A first-class , thoroughly involving B movie",4


In [18]:
Df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14144 entries, 101 to 44018
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Phrase     14144 non-null  object
 1   Sentiment  14144 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 331.5+ KB


In [19]:
Df.describe()

,Sentiment
count,14144.000000
mean,2.000000
std,2.000071
min,0.000000
25%,0.000000
50%,2.000000
75%,4.000000
max,4.000000


In [20]:
# Type of attribute "Content"
type(Df['Phrase'])

#Type of attribute "Title"
# type(Df['Title'])


pandas.core.series.Series

In [21]:
Df

,Phrase,Sentiment
101,would have a hard time sitting through this one,0
103,have a hard time sitting through this one,0
157,Aggressive self-glorification and a manipulati...,0
159,self-glorification and a manipulative whitewash,0
201,Trouble Every Day is a plodding mess .,0
...,...,...
111705,Beautiful to watch,4
52410,"With a cast of A-list Brit actors , it is wort...",4
87417,exceedingly pleasant,4
122146,", Ash Wednesday is suspenseful and ultimately ...",4


In [24]:
Df.Sentiment.value_counts()

0    7072
4    7072
Name: Sentiment, dtype: int64

In [11]:
df1 = pd.concat([
    df[df.Sentiment==0], 
    df[df.Sentiment==4]]) [['Phrase', 'Sentiment' ]]
df1.head()

,Phrase,Sentiment
101,would have a hard time sitting through this one,0
103,have a hard time sitting through this one,0
157,Aggressive self-glorification and a manipulati...,0
159,self-glorification and a manipulative whitewash,0
201,Trouble Every Day is a plodding mess .,0


# Merging All Twelve Functions In One Function

In [47]:
# Writing main function to merge all the preprocessing steps.
def text_preprocessing(text, accented_chars=True, contractions=True, lemmatization_allow = True,
                        extra_whitespace=True, newlines_tabs=True, repeatition=True, 
                       lowercase=True, punctuations=True, mis_spell=True,
                       remove_html=True, links=True,  special_chars=True,
                       stop_words=True):
    """
    This function will preprocess input text and return
    the clean text.
    """
    
    if newlines_tabs == True: #remove newlines & tabs.
        Data = remove_newlines_tabs(text)
        
    if remove_html == True: #remove html tags
        Data = strip_html_tags(Data)
        
    if links == True: #remove links
        Data = remove_links(Data)
        
    if extra_whitespace == True: #remove extra whitespaces
        Data = remove_whitespace(Data)
        
    if accented_chars == True: #remove accented characters
        Data = accented_characters_removal(Data)
        
    if lowercase == True: #convert all characters to lowercase
        Data = lower_casing_text(Data)
        
    if repeatition == True: #Reduce repeatitions   
        Data = reducing_incorrect_character_repeatation(Data)
        
    if contractions == True: #expand contractions
        Data = expand_contractions(Data)
    
    if punctuations == True: #remove punctuations
        Data = removing_special_characters(Data)
    
    stoplist = stopwords.words('english') 
    stoplist = set(stoplist)
    
    if stop_words == True: #Remove stopwords
        Data = removing_stopwords(Data)
        
    spell = Speller(lang='en')
    
    if mis_spell == True: #Check for mis-spelled words & correct them.
        Data = spelling_correction(Data)
        
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
# #     print(Data)
#     if lemmatization_allow == True: #Converts words to lemma form.
#         Data = lemmatization(Data)
    
           
    return Data

In [48]:
Df['Phrase'] = Df['Phrase'].apply(str)

# 1. def remove_newlines_tabs(text)

In [56]:
def remove_newlines_tabs(text):
    """
    This function will remove all the occurrences of newlines, tabs, and combinations like: \\n, \\.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of newlines, tabs, \\n, \\ characters.
        
    Example:
    Input : This is her \\ first day at this place.\n Please,\t Be nice to her.\\n
    Output : This is her first day at this place. Please, Be nice to her. 
    
    """
    
    # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
    Formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')
    return Formatted_text
# len of data :- 1618647 lac words
remove_newlines_tabs('This is her \\ first day at this place.\n Please,\t Be nice to her.\\n')

'This is her   first day at this place.  Please,  Be nice to her. '

# 2. def strip_html_tags(text)

In [59]:
def strip_html_tags(text):
    """ 
    This function will remove all the occurrences of html tags from the text.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of html tags.
        
    Example:
    Input : This is a nice place to live. 
    Output : This is a nice place to live.  
    """
    # Initiating BeautifulSoup object soup.
    soup = BeautifulSoup(text, "html.parser")
    # Get all the text other than html tags.
    stripped_text = soup.get_text(separator=" ")
    return stripped_text
# len of string:- 1616053 lac words

strip_html_tags("This is a <b>nice</b> place to live. ")

'This is a  nice  place to live. '

# 3. def remove_links(text)

In [61]:
def remove_links(text):
    """
    This function will remove all the occurrences of links.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of all types of links.
        
    Example:
    Input : To know more about cats and food & website: catster.com  visit: https://catster.com//how-to-feed-cats
    Output : To know more about cats and food & website: visit:     
    
    """
    
    # Removing all the occurrences of links that starts with https
    remove_https = re.sub(r'http\S+', '', text)
    # Remove all the occurrences of text that ends with .com
    remove_com = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
    return remove_com
# len of words:- 1616053

remove_links('To know more about cats and food & website: catster.com  visit: https://catster.com//how-to-feed-cats')

'To know more about cats and food & website:   visit: '

# 4. def remove_whitespace(text)

In [63]:
def remove_whitespace(text):
    """ This function will remove 
        extra whitespaces from the text
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after extra whitespaces removed .
        
    Example:
    Input : How   are   you   doing   ?
    Output : How are you doing ?     
        
    """
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', text) 
    # There are some instances where there is no space after '?' & ')', 
    # So I am replacing these with one space so that It will not consider two words as one token.
    text = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    return text    
# len of words:- 1596248 lac words 
remove_whitespace('How   are   you   doing   ?')

'How are you doing  ? '

# 5. def accented_characters_removal(text)

In [65]:
# Code for accented characters removal
def accented_characters_removal(text):
    # this is a docstring
    """
    The function will remove accented characters from the 
    text contained within the Dataset.
       
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" with removed accented characters.
        
    Example:
    Input : Málaga, àéêöhello
    Output : Malaga, aeeohello    
        
    """
    # Remove accented characters from text using unidecode.
    # Unidecode() - It takes unicode data & tries to represent it to ASCII characters. 
    text = unidecode.unidecode(text)
    return text
# Len of data:- 1593952 lac of words
accented_characters_removal('Málaga, àéêöhello')

'Malaga, aeeohello'

# 6. def lower_casing_text(text)

In [67]:
# Code for text lowercasing
def lower_casing_text(text):
    
    """
    The function will convert text into lower case.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
         value: text in lowercase
         
    Example:
    Input : The World is Full of Surprises!
    Output : the world is full of surprises!
    
    """
    # Convert text to lower case
    # lower() - It converts all upperase letter of given string to lowercase.
    text = text.lower()
    return text

lower_casing_text('PAkistan Zindabad !')

'pakistan zindabad !'

# 7. def reducing_incorrect_character_repeatation(text)

In [68]:
# Code for removing repeated characters and punctuations

def reducing_incorrect_character_repeatation(text):
    """
    This Function will reduce repeatition to two characters 
    for alphabets and to one character for punctuations.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Finally formatted text with alphabets repeating to 
        two characters & punctuations limited to one repeatition 
        
    Example:
    Input : Realllllllllyyyyy,        Greeeeaaaatttt   !!!!?....;;;;:)
    Output : Reallyy, Greeaatt !?.;:)
    
    """
    # Pattern matching for all case alphabets
    Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
    
    # Limiting all the  repeatation to two characters.
    Formatted_text = Pattern_alpha.sub(r"\1\1", text) 
    
    # Pattern matching for all the punctuations that can occur
    Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
    
    # Limiting punctuations in previously formatted string to only one.
    Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)
    
    # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
    Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)
    return Final_Formatted

reducing_incorrect_character_repeatation('Realllllllllyyyyy,        Greeeeaaaatttt   !!!!?....;;;;:)')


'Reallyy, Greeaatt !?.;:)'

# 8. def expand_contractions(text, contraction_mapping =  CONTRACTION_MAP)

In [70]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
    "would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
}
# The code for expanding contraction words
def expand_contractions(text, contraction_mapping =  CONTRACTION_MAP):
    """expand shortened words to the actual form.
       e.g. don't to do not
    
       arguments:
            input_text: "text" of type "String".
         
       return:
            value: Text with expanded form of shorthened words.
        
       Example: 
       Input : ain't, aren't, can't, cause, can't've
       Output :  is not, are not, cannot, because, cannot have 
    
     """
    # Tokenizing text into tokens.
    list_Of_tokens = text.split(' ')

    # Checking for whether the given token matches with the Key & replacing word with key's value.
    
    # Check whether Word is in lidt_Of_tokens or not.
    for Word in list_Of_tokens: 
        # Check whether found word is in dictionary "Contraction Map" or not as a key. 
         if Word in CONTRACTION_MAP: 
                # If Word is present in both dictionary & list_Of_tokens, replace that word with the key value.
                list_Of_tokens = [item.replace(Word, CONTRACTION_MAP[Word]) for item in list_Of_tokens]
                
    # Converting list of tokens to String.
    String_Of_tokens = ' '.join(str(e) for e in list_Of_tokens) 
    return String_Of_tokens     

expand_contractions("ain't, aren't, can't, cause, can't've")

"ain't, aren't, can't, cause, cannot have"

# 9. def removing_special_characters(text)

In [71]:
# The code for removing special characters
def removing_special_characters(text):
    """Removing all the special characters except the one that is passed within 
       the regex to match, as they have imp meaning in the text provided.
   
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text with removed special characters that don't require.
        
    Example: 
    Input : Hello, K-a-j-a-l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) 
    Output :  Hello, Kajal. This is $100.05 : the payment that you will recieve! Is this okay?
    
   """
    # The formatted text after removing not necessary punctuations.
    Formatted_Text = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', text) 
    # In the above regex expression,I am providing necessary set of punctuations that are frequent in this particular dataset.
    return Formatted_Text

removing_special_characters('')

''

# 10. def removing_stopwords(text)

In [72]:
# The code for removing stopwords
stoplist = stopwords.words('english') 
stoplist = set(stoplist)
def removing_stopwords(text):
    """This function will remove stopwords which doesn't add much meaning to a sentence 
       & they can be remove safely without comprimising meaning of the sentence.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text after omitted all stopwords.
        
    Example: 
    Input : This is Kajal from delhi who came here to study.
    Output : ["'This", 'Kajal', 'delhi', 'came', 'study', '.', "'"] 
    
   """
    # repr() function actually gives the precise information about the string
    text = repr(text)
    # Text without stopwords
    No_StopWords = [word for word in word_tokenize(text) if word.lower() not in stoplist ]
    # Convert list of tokens_without_stopwords to String type.
    words_string = ' '.join(No_StopWords)    
    return words_string

removing_stopwords('This is Kajal from delhi who came here to study.')

"'This Kajal delhi came study . '"

# 11. def spelling_correction(text)

In [73]:
# The code for spelling corrections
def spelling_correction(text):
    ''' 
    This function will correct spellings.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text after corrected spellings.
        
    Example: 
    Input : This is Oberois from Dlhi who came heree to studdy.
    Output : This is Oberoi from Delhi who came here to study.
      
    
    '''
    # Check for spellings in English language
    spell = Speller(lang='en')
    Corrected_text = spell(text)
    return Corrected_text
spelling_correction('This is Oberois from Dlhi who came heree to studdy.')

'This is Oberoi from Delhi who came here to study.'

# 12. def lemmatization(text)

In [74]:
# The code for lemmatization
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatization(text):
    """This function converts word to their root words 
       without explicitely cut down as done in stemming.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text having root words only, no tense form, no plural forms
        
    Example: 
    Input : text reduced 
    Output :  text reduce
    
   """
    # Converting words to their root forms
    lemma = [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]
    return lemma

lemmatization('text reduced')

['text', 'reduce']

In [75]:
# Pre-processing for Content
List_Content = df['Phrase'].to_list()
Final_Article = []
Complete_Content = []
for article in List_Content:
    print(article)
    Processed_Content = text_preprocessing(article) #Cleaned text of Content attribute after pre-processing
    Final_Article.append(Processed_Content)
Complete_Content.extend(Final_Article)
df['Processed_Content'] = Complete_Content

would have a hard time sitting through this one
have a hard time sitting through this one
Aggressive self-glorification and a manipulative whitewash
self-glorification and a manipulative whitewash
Trouble Every Day is a plodding mess .
is a plodding mess
plodding mess
could hate it for the same reason
hate it
hate
is Oedekerk 's realization of his childhood dream to be in a martial-arts flick , and proves that sometimes the dreams of youth should remain just that
baseball movies that try too hard to be mythic
Hampered -- no , paralyzed -- by a self-indulgent script ... that aims for poetry and ends up sounding like satire .
a self-indulgent script
There 's very little sense to what 's going on here ,
avoid
almost feels as if the movie is more interested in entertaining itself than in amusing us .
The movie 's progression into rambling incoherence gives new meaning to the phrase ` fatal script error . '
The movie 's progression into rambling incoherence
gives new meaning to the phrase `

Poor editing , bad bluescreen , and
Poor editing , bad bluescreen
bad bluescreen
completely misses its emotions
failing , ultimately ,
failing , ultimately
There 's a scientific law to be discerned here that producers would be well to heed : Mediocre movies start to drag as soon as the action speeds up ; when the explosions start , they fall to pieces .
There 's a scientific law to be discerned here that producers would be well to heed : Mediocre movies start to drag as soon as the action speeds up ; when the explosions start , they fall to pieces
There 's a scientific law to be discerned here that producers would be well to heed : Mediocre movies start to drag as soon as the action speeds up
stale and uninspired .
and uninspired .
uninspired .
, it wears out its welcome as tryingly as the title character .
could force you to scratch a hole in your head .
could force you to scratch a hole in your head
you to scratch a hole in your head
convoluted
violence
Has no reason to exist , other

small and easily overshadowed by its predictability
The jokes are flat , and the action looks fake .
The jokes are flat ,
Festers in just such a dungpile that you 'd swear you were watching monkeys flinging their feces at you .
Festers in just such a dungpile that you 'd swear you
just such a dungpile
such a dungpile
were watching monkeys flinging their feces at you .
flinging their feces
their feces
In addition to the overcooked , ham-fisted direction , which has all the actors reaching for the back row , the dialogue sounds like horrible poetry .
to the overcooked , ham-fisted direction , which has all the actors reaching for the back row
the overcooked , ham-fisted direction , which has all the actors reaching for the back row
, the dialogue sounds like horrible poetry .
the dialogue sounds like horrible poetry .
sounds like horrible poetry .
horrible
A vile , incoherent mess ... a scummy ripoff of David Cronenberg 's brilliant ` Videodrome . '
A vile , incoherent mess
vile , incohe

a case of two actors who do everything humanly possible to create characters who are sweet and believable , and are defeated by a screenplay that forces them into bizarre , implausible behavior
are defeated by a screenplay that forces them into bizarre , implausible behavior
forces them into bizarre , implausible behavior
-LRB- Carvey 's -RRB- characters are both overplayed and exaggerated , but then again , subtlety has never been his trademark .
-RRB- characters are both overplayed and exaggerated
both overplayed and exaggerated
overplayed and exaggerated
double-barreled rip-off
rip-off
frustratingly refuses to give Pinochet 's crimes a political context
crimes
nightmare
the incessant use
Obvious , obnoxious and didactic burlesque .
Obvious , obnoxious and didactic burlesque
flat as the scruffy sands of its titular community
The young stars are too cute ; the story and ensuing complications are too manipulative ; the message is too blatant ; the resolutions are too convenient
charact

that goes nowhere and goes there very , very slowly
goes nowhere and goes there very , very slowly
A mechanical action-comedy whose seeming purpose is to market the charismatic Jackie Chan to even younger audiences .
Chalk it up as the worst kind of hubristic folly .
up as the worst kind of hubristic folly
the worst kind of hubristic folly
worst kind
worst
stomps in hobnail boots over Natalie Babbitt 's gentle , endearing 1975 children 's novel .
are served with a hack script
is an insultingly inept and artificial examination of grief and its impacts upon the relationships of the survivors .
an insultingly inept and artificial examination of grief and its impacts upon the relationships of the survivors
an insultingly inept and artificial examination of grief and
an insultingly inept and artificial examination of grief
an insultingly inept and artificial examination
insultingly inept and artificial
A tired , unnecessary retread ... a stale copy of a picture that was n't all that great t

...

Drumline ' shows a level of young , Black manhood that is funny , touching , smart and complicated .
You never know where Changing Lanes is going to take you
One of the best rock documentaries ever .
was amused and entertained by the unfolding of Bielinsky 's cleverly constructed scenario , and greatly impressed by the skill of the actors involved in the enterprise
Tackles the difficult subject of grief and loss with such life-embracing spirit
-LRB- Fiji diver Rusi Vulakoro and the married couple Howard and Michelle Hall -RRB- show us the world they love and make us love it , too .
the most beautiful , evocative works
best short
amused and entertained by the unfolding of Bielinsky 's cleverly constructed scenario ,
shine
-LRB- Ferrera -RRB- has the charisma of a young woman who knows how to hold the screen .
Funny , sexy , devastating and incurably romantic
adored
it 's Robert Duvall !
A naturally funny film
an exquisite motion picture in its own right
'll love it and probably want to 

Brian Tufano 's handsome widescreen photography and Paul Grabowsky 's excellent music turn this fairly parochial melodrama into something really rather special .
of the best ensemble casts of the year
it adds up to big box office bucks all but guaranteed .
One of the best looking and stylish
The movie 's captivating details are all in the performances , from Foreman 's barking-mad Taylor to Thewlis 's smoothly sinister Freddie and Bettany\/McDowell 's hard-eyed gangster .
impressive performances
the gorgeous locales and
, passion , and genius
a comic book with soul
is clever and insightful
of what makes Dover Kosashvili 's outstanding feature debut so potent
purely enjoyable
Half Past Dead is just such an achievement .
Fun and nimble .
smart , compelling
a performance that is masterly
the talents of his top-notch creative team
a beautiful , aching sadness
, Cletis Tout is a winning comedy that excites the imagination and tickles the funny bone .
it 's a very entertaining , thought-prov

, well-acted and thought-provoking
Golden Book
talented director
excels as the enigmatic Mika and Anna Mouglalis is a stunning new young talent in one of Chabrol 's most intense psychological mysteries
A positively thrilling combination of ethnography and all the intrigue , betrayal , deceit and murder of a Shakespearean tragedy or a juicy soap opera
A stylish cast and some clever scripting solutions
Dench who really steals the show
of a romantic crime comedy that turns out to be clever , amusing and unpredictable
really good
one of those reputedly `` unfilmable '' novels that has bucked the odds to emerge as an exquisite motion picture in its own right
of impressive talent
tasteful , intelligent manner
avoid many of the condescending stereotypes that so often plague films dealing with the mentally ill
of the most triumphant performances of Vanessa Redgrave 's career
unfolds with all the mounting tension of an expert thriller ,
is a sincerely crafted picture that deserves to emerge fro

If `` Lilo & Stitch '' is n't the most edgy piece of Disney animation to hit the silver screen , then this first film to use a watercolor background since `` Dumbo '' certainly ranks as the most original in years .
one of the best ensemble casts of the year
, The Hot Chick is pretty damned funny .
first-rate , especially Sorvino
It makes compelling , provocative and prescient viewing .
keeps us riveted with every painful nuance , unexpected flashes of dark comedy and the character 's gripping humanity .
so gracefully
to enjoy for a lifetime
it is a kind , unapologetic , sweetheart of a movie , and Mandy Moore leaves a positive impression
the film is also imbued with strong themes of familial ties and spirituality that are powerful and moving without stooping to base melodrama
inspirational status
some very good acting , dialogue ,
wonderfully speculative character
An intelligent and deeply felt work about impossible , irrevocable choices and the price of making them .
rich in emotional

should have a stirring time at this beautifully drawn movie .
to emerge as an exquisite motion picture in its own right
into a genre that has been overexposed , redolent of a thousand cliches , and yet remains uniquely itself , vibrant with originality
is terrific ,
An intelligent , multi-layered and profoundly humanist -LRB- not to mention gently political -RRB- meditation on the values of knowledge , education , and the affects of cultural and geographical displacement .
highly professional film that 's old-fashioned in all the best possible ways
Charming
infuse the rocky path to sibling reconciliation with flashes of warmth and gentle humor
who 's finally been given a part worthy of her considerable talents
an extraordinary poignancy
worm its way into your heart
was made with careful attention to detail and is well-acted by James Spader and Maggie Gyllenhaal
A powerful and telling story that examines forbidden love , racial tension , and other issues that are as valid today as they 

In [ ]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [76]:
df1['input_data'] = df1.Phrase.apply(text_preprocessing)
df1.head()

C:\Users\HP\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


,Phrase,Sentiment,input_data
101,would have a hard time sitting through this one,0,'would hard time sitting one '
103,have a hard time sitting through this one,0,'have hard time sitting one '
157,Aggressive self-glorification and a manipulati...,0,'aggressive self clarification manipulative wh...
159,self-glorification and a manipulative whitewash,0,'self clarification manipulative whitewash '
201,Trouble Every Day is a plodding mess .,0,'trouble every day padding mess . '


In [78]:
df1['input_data'] = df1.Phrase.apply(text_preprocessing)
df1

,Phrase,Sentiment,input_data
101,would have a hard time sitting through this one,0,'would hard time sitting one '
103,have a hard time sitting through this one,0,'have hard time sitting one '
157,Aggressive self-glorification and a manipulati...,0,'aggressive self clarification manipulative wh...
159,self-glorification and a manipulative whitewash,0,'self clarification manipulative whitewash '
201,Trouble Every Day is a plodding mess .,0,'trouble every day padding mess . '
...,...,...,...
155946,is laughingly enjoyable,4,'is laughing enjoyable '
155955,a unique culture that is presented with univer...,4,' unique culture presented universal appeal '
155961,with universal appeal,4,'with universal appeal '
156007,really do a great job of anchoring the charact...,4,'really great job anchoring characters emotion...


In [79]:
Cleaned_Data = Df.to_csv('Cleaned_Data.csv', index = False)